In [1]:
import Foundation
import TensorFlow

In [2]:
let trainCSV = try String(contentsOfFile:"../data/train.csv", encoding: String.Encoding.utf8)
let testCSV = try String(contentsOfFile:"../data/test.csv", encoding: String.Encoding.utf8)

let trainRecords: [[Float]] = trainCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }
let testRecords: [[Float]] = testCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }

let numTrainRecords = trainRecords.count
let numTrainColumns = trainRecords[0].count
let numTestRecords = testRecords.count
let numTestColumns = testRecords[0].count

print(numTrainRecords, numTrainColumns, numTestRecords, numTestColumns)

let xTrain = trainRecords.map{ Array($0[0..<numTrainColumns-1]) }
let yTrain = trainRecords.map{ [$0[numTrainColumns-1]] }
let xTest = testRecords.map{ Array($0[0..<numTestColumns-1]) }
let yTest = testRecords.map{ [$0[numTestColumns-1]] }

print(xTrain.count, xTrain[0].count, yTrain.count, yTrain[0].count,
      xTest.count, xTest[0].count, yTest.count, yTest[0].count)

let xAllTrain = Array(xTrain.joined())
let yAllTrain = Array(yTrain.joined())
let xAllTest = Array(xTest.joined())
let yAllTest = Array(yTest.joined())

print(xAllTrain.count, yAllTrain.count, xAllTest.count, yAllTest.count)

405 14 101 14
405 13 405 1 101 13 101 1
5265 405 1313 101


In [3]:
let XTrain = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numTrainColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTest = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numTestColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


In [4]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

In [5]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [6]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(xTrain.count) / Double(batchSize)))

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [7]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        let batchStart = batch * batchSize
        let batchEnd = min(xTrain.count, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 520.42377, MAE: 20.892487
Epoch 2: MSE: 438.88153, MAE: 18.93572
Epoch 3: MSE: 357.07098, MAE: 16.732164
Epoch 4: MSE: 271.54764, MAE: 14.159134
Epoch 5: MSE: 192.01268, MAE: 11.430877
Epoch 6: MSE: 128.31119, MAE: 8.896483
Epoch 7: MSE: 85.70299, MAE: 7.00624
Epoch 8: MSE: 61.17824, MAE: 5.7895107
Epoch 9: MSE: 47.17949, MAE: 4.9770937
Epoch 10: MSE: 38.07047, MAE: 4.41381
Epoch 11: MSE: 31.953945, MAE: 4.022637
Epoch 12: MSE: 27.899956, MAE: 3.7587492
Epoch 13: MSE: 25.070597, MAE: 3.5508459
Epoch 14: MSE: 23.066021, MAE: 3.3925176
Epoch 15: MSE: 21.514254, MAE: 3.262881
Epoch 16: MSE: 20.257986, MAE: 3.1527746
Epoch 17: MSE: 19.222479, MAE: 3.0617366
Epoch 18: MSE: 18.357973, MAE: 2.9829586
Epoch 19: MSE: 17.650349, MAE: 2.911856
Epoch 20: MSE: 17.034863, MAE: 2.8485107
Epoch 21: MSE: 16.478321, MAE: 2.7929754
Epoch 22: MSE: 15.985419, MAE: 2.7420802
Epoch 23: MSE: 15.546739, MAE: 2.6987376
Epoch 24: MSE: 15.151701, MAE: 2.6606116
Epoch 25: MSE: 14.796304, MAE: 2.62830

Epoch 204: MSE: 4.9603095, MAE: 1.466044
Epoch 205: MSE: 4.937858, MAE: 1.4617198
Epoch 206: MSE: 4.932933, MAE: 1.4599552
Epoch 207: MSE: 4.908646, MAE: 1.4564258
Epoch 208: MSE: 4.897477, MAE: 1.454213
Epoch 209: MSE: 4.8838973, MAE: 1.4508383
Epoch 210: MSE: 4.869089, MAE: 1.448057
Epoch 211: MSE: 4.854481, MAE: 1.446045
Epoch 212: MSE: 4.8369436, MAE: 1.4420056
Epoch 213: MSE: 4.820054, MAE: 1.439647
Epoch 214: MSE: 4.800881, MAE: 1.435232
Epoch 215: MSE: 4.7990246, MAE: 1.4353738
Epoch 216: MSE: 4.7839627, MAE: 1.433239
Epoch 217: MSE: 4.763306, MAE: 1.4284835
Epoch 218: MSE: 4.7491107, MAE: 1.4257692
Epoch 219: MSE: 4.737991, MAE: 1.42338
Epoch 220: MSE: 4.723503, MAE: 1.4212736
Epoch 221: MSE: 4.7049494, MAE: 1.4156811
Epoch 222: MSE: 4.7014866, MAE: 1.4165769
Epoch 223: MSE: 4.6821537, MAE: 1.411845
Epoch 224: MSE: 4.67073, MAE: 1.4111397
Epoch 225: MSE: 4.6540217, MAE: 1.4059249
Epoch 226: MSE: 4.640717, MAE: 1.4049985
Epoch 227: MSE: 4.627718, MAE: 1.4018917
Epoch 228: MSE: 4

Epoch 405: MSE: 2.886117, MAE: 1.0913088
Epoch 406: MSE: 2.8778288, MAE: 1.0924253
Epoch 407: MSE: 2.8784566, MAE: 1.0928522
Epoch 408: MSE: 2.85874, MAE: 1.0863692
Epoch 409: MSE: 2.8574762, MAE: 1.089151
Epoch 410: MSE: 2.85339, MAE: 1.087527
Epoch 411: MSE: 2.840357, MAE: 1.0851805
Epoch 412: MSE: 2.834302, MAE: 1.0812576
Epoch 413: MSE: 2.8359528, MAE: 1.0892751
Epoch 414: MSE: 2.8224182, MAE: 1.0790042
Epoch 415: MSE: 2.8137856, MAE: 1.0829962
Epoch 416: MSE: 2.807865, MAE: 1.0766602
Epoch 417: MSE: 2.7970598, MAE: 1.0762408
Epoch 418: MSE: 2.8012776, MAE: 1.0818405
Epoch 419: MSE: 2.7820601, MAE: 1.0723264
Epoch 420: MSE: 2.7748632, MAE: 1.0756993
Epoch 421: MSE: 2.770912, MAE: 1.074393
Epoch 422: MSE: 2.7612612, MAE: 1.0692304
Epoch 423: MSE: 2.7437658, MAE: 1.0668199
Epoch 424: MSE: 2.7590742, MAE: 1.0736678
Epoch 425: MSE: 2.7415342, MAE: 1.0666776
Epoch 426: MSE: 2.7270854, MAE: 1.066125
Epoch 427: MSE: 2.7332115, MAE: 1.0616734
Epoch 428: MSE: 2.7097654, MAE: 1.0648519
Epoch